In [22]:
import numpy as np
import pandas as pd

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.tree import DecisionTreeClassifier

In [24]:
df = pd.read_csv("./train.csv")

In [25]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [26]:
df = df.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])

In [27]:
xTrain, xTest, yTrain, yTest = train_test_split(df.drop(columns='Survived'), df['Survived'], test_size=0.1)

In [28]:
xTrain

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
106,3,female,21.0,0,0,7.6500,S
444,3,male,NaN,0,0,8.1125,S
110,1,male,47.0,0,0,52.0000,S
547,2,male,NaN,0,0,13.8625,C
625,1,male,61.0,0,0,32.3208,S
...,...,...,...,...,...,...,...
782,1,male,29.0,0,0,30.0000,S
645,1,male,48.0,1,0,76.7292,C
739,3,male,NaN,0,0,7.8958,S
204,3,male,18.0,0,0,8.0500,S


# now we need to design different column transformers

In [29]:
# Imputation Transformer
t1 = ColumnTransformer(
    [
        ('ageTransformer', SimpleImputer(), [2]),
        ('stationTransformer', SimpleImputer(strategy='most_frequent'), [6])
    ],
    remainder='passthrough'
)

In [30]:
# Encoder Column Transformer
t2 = ColumnTransformer(
    [('setTransformer', OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first'), [1,6])],
    remainder='passthrough'
)

In [31]:
# Scaler Transformer
t3 = ColumnTransformer(
    [
        ('dataScaler', MinMaxScaler(), slice(0,8))
    ],
    remainder='passthrough'
)

In [32]:
# Feature Selection
t4 = SelectKBest(score_func=chi2, k=7)

In [33]:
# Model Training
t5 = DecisionTreeClassifier()

# Now we need to build the pipeline

In [34]:
pipe = Pipeline([
    ('Transformer1', t1),
    ('Transformer2', t2),
    ('Transformer3', t3),
    ('Transformer4', t4),
    ('Transformer5', t5)
])

In [39]:
pipiFast = make_pipeline(t1, t2, t3, t4, t5)

In [40]:
# Training the model
pipiFast.fit(xTrain, yTrain)

/Users/maverick/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ValueError: could not convert string to float: 'female'

In [41]:
pipe.predict(xTest)

/Users/maverick/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


NotFittedError: This SelectKBest instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.